In [63]:
import os
import sys
import numpy as np
import deepdish as dd
import pandas as pd
from argparse import Namespace

In [7]:
from gwpopulation_pipe.data_collection import load_gwtc_1_events



args = Namespace(gwtc1_samples_regex='/home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW*.hdf5')

posteriors = load_gwtc_1_events(args)


16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW150914_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW151012_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW151226_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170104_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170608_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170729_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170809_GWTC-1.hdf5.
16:53 bilby INFO    : Loaded Overall from /home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW170814_GWTC-1.hdf5.
16:53 bi

NameError: name 'postertior' is not defined

In [9]:
posteriors['/home/shanika.galaudage/O3/population/GWTC-1_sample_release/GW150914_GWTC-1.hdf5']

,redshift,mass_1,a_1,cos_tilt_1,mass_2,a_2,cos_tilt_2,mass_ratio
0,0.108213,35.225515,0.417147,-0.280624,33.427289,0.867740,0.403853,0.948951
1,0.085328,31.898266,0.125709,-0.757349,31.496839,0.260679,-0.312285,0.987415
2,0.078865,35.124252,0.581047,0.649781,31.487262,0.926893,-0.510843,0.896454
3,0.082373,33.641806,0.235808,0.116578,32.968738,0.094391,-0.720505,0.979993
4,0.073895,36.760824,0.511521,-0.438237,29.467517,0.868009,0.269333,0.801601
...,...,...,...,...,...,...,...,...
8345,0.066089,35.233424,0.484003,0.194507,31.288003,0.627191,-0.408345,0.888020
8346,0.097492,34.464289,0.589654,-0.737792,33.416243,0.650758,0.875384,0.969590
8347,0.094773,34.768191,0.708407,0.852085,32.662388,0.714805,-0.797475,0.939433
8348,0.116737,40.192456,0.389284,-0.251461,28.289953,0.521304,0.830526,0.703862


In [89]:
import pandas as pd
import glob
import tqdm
import re

SAMPLE_SRC = "/home/avi.vajpeyi/projects/agn_phenomenological_model/data/gwtc_samples/"
PARAM_TO_KEEP = ["a_1", "a_2", "cos_tilt_1", "cos_tilt_2", "mass_ratio", "mass_1_source", "mass_2_source", "redshift", "cos_theta_12", "chi_p", "chi_eff"]

posterior_files = [f for f in glob.glob(SAMPLE_SRC+"*.csv") if "prior" not in f]
prior_files = glob.glob(SAMPLE_SRC + "*_prior.csv")

reject_events = ["GW170817", "S190425z", "S190426c", "S190719an", "190814", "S190909w", "S190930ak"]
posts =[]
events =[]
def get_event_name(fname):
    return re.findall(r"(\w*\d{6}[a-z]*)", fname)[0]

def add_agn_samples_to_df(df):
    df['s1x'], df['s1y'], df['s1z'] = df['spin_1x'], df['spin_1y'], df['spin_1z']
    df['s2x'], df['s2y'], df['s2z'] = df['spin_2x'], df['spin_2y'], df['spin_2z']
    df['s1_dot_s2'] = (df['s1x'] * df['s2x']) + (df['s1y'] * df['s2y']) + (
            df['s1z'] * df['s2z'])
    df['s1_mag'] = np.sqrt(
        (df['s1x'] * df['s1x']) + (df['s1y'] * df['s1y']) + (df['s1z'] * df['s1z']))
    df['s2_mag'] = np.sqrt(
        (df['s2x'] * df['s2x']) + (df['s2y'] * df['s2y']) + (df['s2z'] * df['s2z']))
    df['cos_theta_12'] = df['s1_dot_s2'] / (df['s1_mag'] * df['s2_mag'])
    return df


def read_posteriors(posterior_files):
    # posterior_files = [f for f in glob.glob(SAMPLE_SRC+"*.csv") if "prior" not in f]
    posteriors = {}
    for f in tqdm.tqdm(posterior_files, desc='Posteriors'):
        event = get_event_name(f)
        if event not in reject_events:
            d = pd.read_csv(f,sep=" ", index_col=False)
            try:
                d = add_agn_samples_to_df(d)
                posteriors.update({f:d[PARAM_TO_KEEP]})
            except Exception as e:
                print(f"Skipping {event}: {e}\n{d.columns.values.tolist()}")
    return posteriors



    
posteriors =  read_posteriors(posterior_files)
# priors = {f:pd.read_csv(f,sep=" ", index_col=False) for f in tqdm.tqdm(prior_files, desc="Priors")}


agn_samples = "/home/avi.vajpeyi/projects/agn_phenomenological_model/data/gwtc_agn_samples/" 
# os.mkdir(agn_samples)

for fname, posterior in posteriors.items():
    new_f = os.path.join(agn_samples, os.path.basename(fname))
    posterior.to_csv(new_f, index=False, sep=' ')

Posteriors:  31%|███       | 27/88 [00:01<00:04, 15.18it/s]

Skipping GW170729: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  50%|█████     | 44/88 [00:03<00:02, 14.71it/s]

Skipping GW170814: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  56%|█████▌    | 49/88 [00:03<00:02, 15.72it/s]

...

Posteriors:  90%|████████▉ | 79/88 [00:05<00:00, 17.98it/s]

Skipping GW170818: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']
Skipping GW170809: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors:  95%|█████████▌| 84/88 [00:05<00:00, 17.78it/s]

Skipping GW170608: 'spin_1x'
['cos_theta_jn,luminosity_distance,ra,dec,mass_1,mass_2,a_1,a_2,cos_tilt_1,cos_tilt_2,log_likelihood']


Posteriors: 100%|██████████| 88/88 [00:05<00:00, 14.94it/s]


In [62]:
from copy import deepcopy
import matplotlib
matplotlib.rcParams["text.usetex"] = False  # noqa
import matplotlib.pyplot as plt

for filename in posteriors.keys():
    event = get_event_name(filename)
    if event not in reject_events:
        posts.append(posteriors[filename])
        events.append(event)

        
def plot_summary(posteriors: list, events: list):
    posteriors = deepcopy(posteriors)
    plot_dir = os.path.join("outdir/data")
    plot_chi = True
    for posterior in posteriors:
        if all(
            [
                key in posterior
                for key in ["a_1", "a_2", "cos_tilt_1", "cos_tilt_2", "mass_ratio"]
            ]
        ):
            posterior["chi_eff"] = (
                posterior["a_1"] * posterior["cos_tilt_1"]
                + posterior["a_2"] * posterior["cos_tilt_2"] * posterior["mass_ratio"]
            ) / (1 + posterior["mass_ratio"])
            posterior["chi_p"] = np.maximum(
                posterior["a_1"] * (1 - posterior["cos_tilt_1"] ** 2) ** 0.5,
                posterior["mass_ratio"]
                * (3 + 4 * posterior["mass_ratio"])
                / (4 + 3 * posterior["mass_ratio"])
                * posterior["a_2"]
                * (1 - posterior["cos_tilt_2"] ** 2) ** 0.5,
            )
        else:
            plot_chi = False
    plot_mass_2 = True
    for posterior in posteriors:
        try:
            posterior["mass_2"] = posterior["mass_1"] * posterior["mass_ratio"]
        except KeyError:
            plot_mass_2 = False
    print(f"Plotting params {posteriors[0].keys()}")
    for parameter in posteriors[0].keys():
        if parameter in ["chi_eff", "chi_p"] and not plot_chi:
            continue
        elif parameter == "mass_2" and not plot_mass_2:
            continue
        print(f"Making box plot for {parameter}")
        file_name = os.path.join(plot_dir, parameter)
        data = [post[parameter] for post in posteriors]
        fig = plt.figure(figsize=(len(posteriors), 5))
        plt.violinplot(data)
        plt.ylabel(parameter.replace("_", " "))
        plt.xticks(np.arange(1, len(events) + 1), events, rotation=90)
        if parameter == "prior":
            plt.yscale("log")
        plt.tight_layout()
        plt.savefig(file_name + ".png")
        plt.close(fig)
        
plot_summary(posts, events)

Making box plot for a_1
Making box plot for a_2
Making box plot for cos_tilt_1
Making box plot for cos_tilt_2
Making box plot for mass_ratio
Making box plot for mass_1
Making box plot for mass_2
Making box plot for redshift
Making box plot for cos_theta_12
Making box plot for chi_p
Making box plot for chi_eff


In [78]:
"""Hacky script to grab the "PublicationSamples" and priors from the GWTC1+2 cat
Note: runtime ~2hrs
"""
import glob
import os
import sys
import traceback

import numpy as np
import pandas as pd
import pesummary
import tqdm
from pesummary.io import read

MAX_NUM_SAMPLES = 5000
GWTC2_SOURCE = "/home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/*.h5"
GWTC1_SOURCE = "/home/shanika.galaudage/O3/population/GWTC-1_sample_release/*.hdf5"
OUTDIR = "../data/gwtc_samples"


def safe_run(func):
    def func_wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            tb = traceback.format_tb(exc_tb)
            tb = "||".join(tb)
            print(
                f"ERROR: {exc_type}\n" f"Traceback: [{tb}]\n" f"Message: {e}"
            )
            return None

    return func_wrapper


@safe_run
def process_gwtc2_file(res_file, outdir):
    event_name = os.path.basename(res_file).split(".h")[0]
    samp_fname = f"{outdir}/{event_name}.csv"
    prior_fname = f"{outdir}/{event_name}_prior.csv"
    print(f"Checking fname {samp_fname}")
    if not (os.path.isfile(samp_fname)):
        print(f"Opening {res_file}")
        data = pesummary.io.read(res_file)
        if len(data.samples) > MAX_NUM_SAMPLES:
            data.downsample(MAX_NUM_SAMPLES)
        samples_dict = data.samples_dict
        samples = pd.DataFrame(samples_dict['PublicationSamples'])
        samples.to_csv(samp_fname, sep=' ', index=False)
        priors_array = np.load(res_file.replace(".h5", "_prior.npy"))
        priors = pd.DataFrame(priors_array).sample(MAX_NUM_SAMPLES)
        priors.to_csv(prior_fname, sep=' ', index=False)


@safe_run
def process_gwtc1_file(res_file, outdir):
    event_name = os.path.basename(res_file).split(".h")[0].split("_")[0]
    posterior_filename = f"{outdir}/{event_name}.csv"
    prior_filename = f"{outdir}/{event_name}_prior.csv"
    print(f"Checking fname {posterior_filename}")
    if not (os.path.isfile(posterior_filename)):
        print(f"Opening {res_file}")
        data = pesummary.io.read(res_file)
        if len(data.samples) > MAX_NUM_SAMPLES:
            data.downsample(MAX_NUM_SAMPLES)
        parameters = data.parameters
        posterior_samples = data.samples
        prior_samples = pd.DataFrame(data.priors['samples']).sample(MAX_NUM_SAMPLES)
        prior_samples = prior_samples.to_numpy()
        kwargs = dict(parameters=parameters, file_format="csv")
        if not os.path.isfile(posterior_filename):
            pesummary.io.write(samples=posterior_samples,
                               filename=posterior_filename, **kwargs)
        if not os.path.isfile(prior_filename):
            pesummary.io.write(samples=prior_samples, filename=prior_filename,
                               **kwargs)


def main():
    gwtc1_files = glob.glob(GWTC1_SOURCE)
    gwtc2_files = glob.glob(GWTC2_SOURCE)
    gwtc2_files = [f for f in gwtc2_files if "comoving" not in f]
    os.makedirs(OUTDIR, exist_ok=True)
    print("Processing GWTC1")
    for f in tqdm.tqdm(gwtc1_files):
        process_gwtc1_file(f, OUTDIR)
    print("Processing GWTC2")
    for f in tqdm.tqdm(gwtc2_files):
        process_gwtc2_file(f, OUTDIR)



In [81]:
OUTDIR = "../data/test"
os.makedirs(OUTDIR, exist_ok=True)
process_gwtc2_file('/home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/GW190521.h5',OUTDIR)

Checking fname ../data/test/GW190521.csv
Opening /home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/GW190521.h5


In [86]:
df = pd.read_csv("~/projects/agn_phenomenological_model/data/test/GW190521.csv", sep=' ')
df.columns.values

array(['log_likelihood', 'chi_eff', 'total_mass_source',
       'chi_p', 'spin_1y', 'final_mass', 'phi_2', 'phi_12',
       'ra', 'phi_jl', 'tilt_2', 'cos_theta_jn', 'a_2',
       'peak_luminosity', 'mass_1', 'mass_ratio', 'cos_tilt_1',
       'a_1', 'final_mass_source', 'phase',
       'inverted_mass_ratio', 'radiated_energy', 'mass_2',
       'mass_2_source', 'dec', 'psi', 'total_mass', 'theta_jn',
       'redshift', 'iota', 'spin_1x', 'chirp_mass', 'cos_iota',
       'comoving_distance', 'spin_2y', 'tilt_1', 'spin_2z',
       'mass_1_source', 'geocent_time', 'cos_tilt_2',
       'final_spin', 'luminosity_distance', 'spin_1z',
       'chirp_mass_source', 'phi_1', 'symmetric_mass_ratio',
       'spin_2x'], dtype=object)

In [88]:
df['mass_1_source'].describe()

count    59466.000000
mean        93.702182
std         17.534144
min         60.378589
25%         82.796044
50%         91.003491
75%        100.600575
max        300.083940
Name: mass_1_source, dtype: float64